## Import Libraries

In [1]:
# Common imports
import os
import numpy as np

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# TensorFlow imports
# may differs from version to versions

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.preprocessing import image

## Set Parameter

In [8]:
# Dataset information

# Test dataset is set explicitly, because the amount of data is very small
train_image_folder = os.path.join('Dataset', 'train')
test_image_folder = os.path.join('Dataset', 'test')
img_height, img_width = 250, 250  # size of images
num_classes = 2  # Tedy - Unknown

# Training settings
validation_ratio = 0.10 # 10% for the validation
batch_size = 16

AUTOTUNE = tf.data.AUTOTUNE

## Create Dataset

### Read dataset from folders

In [9]:
# Train and validation sets
train_ds = keras.preprocessing.image_dataset_from_directory(
    train_image_folder,
    validation_split=validation_ratio,
    subset="training",
    seed=42,
    image_size=(img_height, img_width),
    label_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

val_ds = keras.preprocessing.image_dataset_from_directory(
    train_image_folder,
    validation_split=validation_ratio,
    subset="validation",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=True)

Found 180 files belonging to 2 classes.
Using 162 files for training.
Found 180 files belonging to 2 classes.
Using 18 files for validation.


In [10]:
# Test set
test_ds = keras.preprocessing.image_dataset_from_directory(
    test_image_folder,
    image_size=(img_height, img_width),
    label_mode='categorical',
    shuffle=False)

Found 20 files belonging to 2 classes.


In [11]:
class_names = test_ds.class_names
class_names

['Tedy', 'Unknown']

# Build The Model (ResNet50)

In [12]:
base_model = keras.applications.ResNet50(weights='imagenet',
                                        include_top=False,  # without dense part of the network
                                        input_shape=(img_height, img_width, 3))

In [13]:
# Set layers to non-trainable
for layer in base_model.layers:
    layer.trainable = False

In [14]:
# Add custom layers on top of ResNet
global_avg_pooling = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(num_classes, activation='sigmoid')(global_avg_pooling)

face_classifier = keras.models.Model(inputs=base_model.input,
                                    outputs=output,
                                    name='ResNet50')
face_classifier.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 256, 256, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 125, 125, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 125, 125, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [15]:
# ModelCheckpoint to save model in case of interrupting the learning process
checkpoint = ModelCheckpoint("Model/classifier_resnet50.h5",
                            monitor="val_loss",
                            mode="min",
                            save_best_only=True,
                            verbose=1)

# EarlyStopping to find best model with a large number of epochs
earlystop = EarlyStopping(monitor='val_loss',
                        restore_best_weights=True,
                        patience=3,  # number of epochs with no improvement after which training will be stopped
                        verbose=1)

callbacks = [earlystop, checkpoint]

In [16]:
face_classifier.compile(loss='categorical_crossentropy',
                        optimizer=keras.optimizers.Adam(learning_rate=0.01),
                        metrics=['accuracy'])

## Training

In [17]:
epochs = 20

In [18]:
history = face_classifier.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds)

face_classifier.save("Model/classifier_resnet50.h5")

Epoch 1/20
11/11 [==============================] - 44s 3s/step - loss: 0.0988 - accuracy: 0.9568 - val_loss: 1.8080e-06 - val_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.00000, saving model to Model\classifier_resnet50.h5


C:\Users\hello\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/20
11/11 [==============================] - 35s 3s/step - loss: 0.0199 - accuracy: 0.9938 - val_loss: 3.9736e-08 - val_accuracy: 1.0000

Epoch 00002: val_loss improved from 0.00000 to 0.00000, saving model to Model\classifier_resnet50.h5
Epoch 3/20
11/11 [==============================] - 36s 3s/step - loss: 2.4150e-05 - accuracy: 1.0000 - val_loss: 1.0324e-05 - val_accuracy: 1.0000

Epoch 00003: val_loss did not improve from 0.00000
Epoch 4/20
11/11 [==============================] - 34s 3s/step - loss: 6.8553e-05 - accuracy: 1.0000 - val_loss: 6.7331e-05 - val_accuracy: 1.0000

Epoch 00004: val_loss did not improve from 0.00000
Epoch 5/20
11/11 [==============================] - 34s 3s/step - loss: 1.5265e-04 - accuracy: 1.0000 - val_loss: 9.9830e-05 - val_accuracy: 1.0000
Restoring model weights from the end of the best epoch.

Epoch 00005: val_loss did not improve from 0.00000
Epoch 00005: early stopping


## Testing

In [23]:
def test_image_classifier_with_folder(model, path, y_true, img_height=250, img_width=250, class_names=['Tedy', 'Unknown']):
    '''
    Read all images from 'path' using tensorflow.keras.preprocessing.image module, 
    than classifies them using 'model' and compare result with 'y_true'.
    Calculate total accuracy based on 'path' test set.

    Parameters:
        model : Image classifier
        path (str): Path to the folder with images you want to test classifier on 
        y_true : True label of the images in the folder. Must be in 'class_names' list
        img_height (int): The height of the image that the classifier can process 
        img_width (int): The width of the image that the classifier can process
        class_names (array-like): List of class names 

    Returns:
        None
    '''
    num_classes = len(class_names)  # Number of classes
    total = 0  # number of images total
    correct = 0  # number of images classified correctly

    for filename in os.listdir(path):
        # read each image in the folder and classifies it
        test_path = os.path.join(path, filename)
        test_image = image.load_img(
            test_path, target_size=(img_height, img_width, 3))
        # from image to array, can try type(test_image)
        test_image = image.img_to_array(test_image)
        # shape from (250, 250, 3) to (1, 250, 250, 3)
        test_image = np.expand_dims(test_image, axis=0)
        result = model.predict(test_image)

        y_pred = class_names[np.array(result[0]).argmax(
            axis=0)]  # predicted class
        iscorrect = 'correct' if y_pred == y_true else 'incorrect'
        print('{} - {}'.format(iscorrect, filename))
        for index in range(num_classes):
            print("\t{:6} with probabily of {:.2f}%".format(
                class_names[index], result[0][index] * 100))

        total += 1
        if y_pred == y_true:
            correct += 1

    print("\nTotal accuracy is {:.2f}% = {}/{} samples classified correctly".format(
        correct/total*100, correct, total))

In [24]:
model_name = 'classifier_resnet50.h5'
face_classifier = keras.models.load_model(f'Model/{model_name}')

In [26]:
test_image_classifier_with_folder(face_classifier,
                                'Dataset/test/Tedy',
                                y_true='Tedy')

incorrect - Screenshot_100.png
	Tedy   with probabily of 53.72%
	Unknown with probabily of 76.47%
incorrect - Screenshot_101.png
	Tedy   with probabily of 47.43%
	Unknown with probabily of 81.86%
correct - Screenshot_102.png
	Tedy   with probabily of 98.20%
	Unknown with probabily of 5.28%
correct - Screenshot_103.png
	Tedy   with probabily of 100.00%
	Unknown with probabily of 0.00%
correct - Screenshot_104.png
	Tedy   with probabily of 100.00%
	Unknown with probabily of 0.00%
correct - Screenshot_95.png
	Tedy   with probabily of 100.00%
	Unknown with probabily of 0.00%
correct - Screenshot_96.png
	Tedy   with probabily of 100.00%
	Unknown with probabily of 0.00%
correct - Screenshot_97.png
	Tedy   with probabily of 100.00%
	Unknown with probabily of 0.00%
correct - Screenshot_98.png
	Tedy   with probabily of 100.00%
	Unknown with probabily of 0.00%
correct - Screenshot_99.png
	Tedy   with probabily of 100.00%
	Unknown with probabily of 0.00%

Total accuracy is 80.00% = 8/10 samples c

In [27]:
test_image_classifier_with_folder(face_classifier,
                                'Dataset/test/Unknown',
                                y_true='Unknown')

correct - Abdullah_Ahmad_Badawi_0001.jpg
	Tedy   with probabily of 0.00%
	Unknown with probabily of 100.00%
correct - Abdullah_al-Attiyah_0001.jpg
	Tedy   with probabily of 0.00%
	Unknown with probabily of 100.00%
correct - Abdullah_Gul_0001.jpg
	Tedy   with probabily of 0.00%
	Unknown with probabily of 100.00%
correct - Abdullah_Nasseef_0001.jpg
	Tedy   with probabily of 0.00%
	Unknown with probabily of 100.00%
correct - Abdullatif_Sener_0001.jpg
	Tedy   with probabily of 0.00%
	Unknown with probabily of 100.00%
correct - Abel_Aguilar_0001.jpg
	Tedy   with probabily of 0.00%
	Unknown with probabily of 100.00%
correct - Abel_Pacheco_0001.jpg
	Tedy   with probabily of 0.00%
	Unknown with probabily of 100.00%
correct - Abid_Hamid_Mahmud_Al-Tikriti_0001.jpg
	Tedy   with probabily of 0.00%
	Unknown with probabily of 100.00%
correct - Abner_Martinez_0001.jpg
	Tedy   with probabily of 0.00%
	Unknown with probabily of 100.00%
correct - Abraham_Foxman_0001.jpg
	Tedy   with probabily of 0.00%
	